# Location of parameters
* **Electron density** $n_e$: The number of electrons per unit volume in the plasma.
* **Electorn temperature** $T_e$: The temperature of the electrons in the plasma.
* **Radiated power (bolometers)** $P_{rad}$: The power radiated by the plasma.
* **Internal inductance** $l_i$: A measure of the plasma's inductance.
* **plasma current** $I_p$: The flow of charged particles in the plasma.

In [ ]:
import zarr
import s3fs
import matplotlib.pyplot as plt
import xarray as xr
import dask
import numpy as np                                                      
import fsspec

## Data in a single shot
Let us take shot number 30420.
### Level 1 file.
Download using the following command:
```bash
./s5cmd --no-sign-request --endpoint-url https://s3.echo.stfc.ac.uk cp "s3://mast/level1/shots/30420.zarr/*" ./30420.zarr/level1/
```

In [15]:
!ls "/home/hell/Desktop/UKAEA/Projects/1. 2024/1. Dec/1. Disruptions/bulk download/30420.zarr"

abm  aga  air  ama  amh  ane  arp  asx	efm  rba  rca  rgc  xdc  xma  xpc
ada  ahx  ait  amb  amm  anu  asb  ayc	esm  rbb  rco  rir  xho  xmb  xsx
adg  aim  alp  amc  ams  aoe  asm  aye	esx  rbc  rgb  rit  xim  xmo


In [19]:
!tree "/home/hell/Desktop/UKAEA/Projects/1. 2024/1. Dec/1. Disruptions/bulk download/30420.zarr" -L 2

/home/hell/Desktop/UKAEA/Projects/1. 2024/1. Dec/1. Disruptions/bulk download/30420.zarr
├── abm
│   ├── calib_shot
│   ├── channel_status
│   ├── channel_type
│   ├── chord
│   ├── dim_0
│   ├── gain
│   ├── i-bol
│   ├── km
│   ├── passnumber
│   ├── phi-chords
│   ├── phi-slits
│   ├── prad_pol
│   ├── rbm
│   ├── r-chords
│   ├── r-slits
│   ├── r_tco_emis
│   ├── status
│   ├── taum
│   ├── tco_emis
│   ├── time
│   ├── v-bol
│   ├── version
│   ├── z-chords
│   └── z-slits
├── ada
│   ├── dalpha_integrated
│   ├── dalpha_inverted
│   ├── dalpha_peak_radius
│   ├── dalpha_raw_full
│   ├── dim_0
│   ├── dim_1
│   ├── geo_full
│   ├── inner_integrated
│   ├── inner_peak_radius
│   ├── passnumber
│   ├── status
│   └── time
├── adg
│   ├── density_gradient
│   ├── gradient_position
│   ├── passnumber
│   ├── status
│   └── time
├── aga
│   ├── bc11
│   ├── bc5
│   ├── fig
│   ├── fig_raw
│   ├── hecc
│   ├── helios
│   ├── hl1
│   ├── hl11
│   ├── hm12
│   ├── hu11
│   ├── hu6
│   ├─

## Explore amx, ayc, atm
* `amc` is the plasma current data.
* `ayc` is the Thomspon scattering data.
* `ane` is CO2 interferometer data.
* `efm` is equilibrium reconstruction data (EFIT).
* `xmo` is the magnetic data example Mirnov coils.
* `rba`is Photron bullet camera A.
* `rbb` is Photron bullet camera B.

In [11]:
# Set up the S3 storage options
shot_number = 30420
endpoint = "https://s3.echo.stfc.ac.uk"
fs = s3fs.S3FileSystem(endpoint_url=endpoint, anon=True)

filename = "s3://mast/level1/shots/"+ str(shot_number) + ".zarr"

In [6]:

datasets = {
    "amc": filename + "/amc",
    "ayc": filename + "/ayc",
    "atm": filename + "/atm"
}

loaded_datasets = {}

for name, path in datasets.items():
    try:
        loaded_datasets[name] = xr.open_zarr(fs.get_mapper(path))
        print(f"Successfully loaded: {name}")
    except Exception as e:
        print(f"Error loading '{name}': {e}")

Successfully loaded: amc
Successfully loaded: ayc
Error loading 'atm': No such file or directory: '<fsspec.mapping.FSMap object at 0x7bc972488250>'


In [8]:
loaded_datasets["amc"]

<xarray.Dataset> Size: 5MB
Dimensions:            (time: 30000)
Coordinates:
  * time               (time) float32 120kB -2.0 -2.0 -2.0 ... 3.999 4.0 4.0
Data variables: (12/46)
    efps_current       (time) float32 120kB dask.array<chunksize=(30000,), meta=np.ndarray>
    error_field_02     (time) float32 120kB dask.array<chunksize=(30000,), meta=np.ndarray>
    error_field_05     (time) float32 120kB dask.array<chunksize=(30000,), meta=np.ndarray>
    p2il_coil_current  (time) float32 120kB dask.array<chunksize=(30000,), meta=np.ndarray>
    p2il_feed_current  (time) float32 120kB dask.array<chunksize=(30000,), meta=np.ndarray>
    p2iu_coil_current  (time) float32 120kB dask.array<chunksize=(30000,), meta=np.ndarray>
    ...                 ...
    p6u_current        (time) float32 120kB dask.array<chunksize=(30000,), meta=np.ndarray>
    plasma_current     (time) float32 120kB dask.array<chunksize=(30000,), meta=np.ndarray>
    sol_current        (time) float32 120kB dask.array<chunksize=(30000,), meta=np.ndarray>
    status             float32 4B ...
    tf_current         (time) float32 120kB dask.array<chunksize=(30000,), meta=np.ndarray>
    version            float32 4B ...
Attributes:
    description:  Plasma Current and PF/TF Coil Currents
    file_name:    amc0304.20
    format:       IDA3
    mds_name:     None
    name:         amc
    quality:      Not Checked
    shot_id:      30420
    signal_type:  Analysed
    source:       amc
    uda_name:     AMC
    uuid:         01aad0c4-2a84-59e2-8b1b-168b4bd66aa3
    version:      0

In [9]:
loaded_datasets["ayc"]

<xarray.Dataset> Size: 1MB
Dimensions:              (acqiris_time: 73, radial_index: 131, arb: 130,
                          time: 73, spectral_index: 4, instrument_time: 20,
                          time_segment: 73, xyc_time: 73)
Coordinates:
  * acqiris_time         (acqiris_time) float32 292B 0.0 1.0 2.0 ... 71.0 72.0
  * arb                  (arb) float32 520B 0.0 1.0 2.0 ... 127.0 128.0 129.0
  * instrument_time      (instrument_time) float32 80B 1.0 2.0 3.0 ... 19.0 20.0
  * radial_index         (radial_index) float32 524B 0.0 1.0 2.0 ... 129.0 130.0
  * spectral_index       (spectral_index) float32 16B 1.0 2.0 3.0 4.0
  * time                 (time) float32 292B 0.004166 0.008332 ... 0.3 0.3042
  * xyc_time             (xyc_time) float32 292B 0.0 1.0 2.0 ... 70.0 71.0 72.0
Dimensions without coordinates: time_segment
Data variables: (12/36)
    acqiris_time_        (acqiris_time) float32 292B dask.array<chunksize=(73,), meta=np.ndarray>
    angle                (radial_index) float32 524B dask.array<chunksize=(131,), meta=np.ndarray>
    aspectra             (time, radial_index, spectral_index) float32 153kB dask.array<chunksize=(73, 131, 4), meta=np.ndarray>
    chi2                 (time, radial_index) float32 38kB dask.array<chunksize=(73, 131), meta=np.ndarray>
    covariance_ne_te     (time, radial_index) float32 38kB dask.array<chunksize=(73, 131), meta=np.ndarray>
    instrument_dr        (instrument_time, radial_index) float32 10kB dask.array<chunksize=(20, 131), meta=np.ndarray>
    ...                   ...
    te_error             (time, radial_index) float32 38kB dask.array<chunksize=(73, 131), meta=np.ndarray>
    version_fibre        float32 4B ...
    version_poly         float32 4B ...
    version_raman        float32 4B ...
    xyc_time_            (xyc_time) float32 292B dask.array<chunksize=(73,), meta=np.ndarray>
    yag_nelint           (time) float32 292B dask.array<chunksize=(73,), meta=np.ndarray>
Attributes:
    description:  Core Thomson scattering data
    file_name:    ayc0304.20
    format:       IDA3
    mds_name:     None
    name:         ayc
    quality:      Not Checked
    shot_id:      30420
    signal_type:  Analysed
    source:       ayc
    uda_name:     AYC
    uuid:         8d043ece-8bf8-5af8-87e4-d2a1b01716fa
    version:      0

In [10]:
loaded_datasets["atm"]

KeyError: 'atm'

## Grab useful parameters
* **Electron density** $n_e$: The number of electrons per unit volume in the plasma.
* **Electron temperature** $T_e$: The temperature of the electrons in the plasma.
* **Radiated power (bolometers)** $P_{rad}$: The power radiated by the plasma.
* **Internal inductance** $l_i$: A measure of the plasma's inductance.
* **plasma current** $I_p$: The flow of charged particles in the plasma.

In [16]:
# plasma current
loaded_datasets["amc"]["plasma_current"]

<xarray.DataArray 'plasma_current' (time: 30000)> Size: 120kB
dask.array<open_dataset-plasma_current, shape=(30000,), dtype=float32, chunksize=(30000,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) float32 120kB -2.0 -2.0 -2.0 -1.999 ... 3.999 3.999 4.0 4.0
Attributes: (12/18)
    description:  Plasma Current
    dims:         ['time']
    file_name:    None
    format:       None
    label:        Plasma Current
    mds_name:     \TOP.ANALYSED.AMC.PLASMA:CURRENT
    ...           ...
    source:       amc
    time_index:   0
    uda_name:     AMC_PLASMA CURRENT
    units:        kA
    uuid:         6327e756-e587-5c74-8aed-4e595d3546cf
    version:      0

In [17]:
# Electron density
loaded_datasets["ayc"]["ne"]

<xarray.DataArray 'ne' (time: 73, radial_index: 131)> Size: 38kB
dask.array<open_dataset-ne, shape=(73, 131), dtype=float32, chunksize=(73, 131), chunktype=numpy.ndarray>
Coordinates:
  * radial_index  (radial_index) float32 524B 0.0 1.0 2.0 ... 128.0 129.0 130.0
  * time          (time) float32 292B 0.004166 0.008332 0.0125 ... 0.3 0.3042
Attributes: (12/18)
    description:  Electron Density
    dims:         ['time', 'radial_index']
    file_name:    None
    format:       None
    label:        electron density
    mds_name:     \TOP.ANALYSED.AYC:NE
    ...           ...
    source:       ayc
    time_index:   0
    uda_name:     AYC_NE
    units:        1 / m ** 3
    uuid:         0141808d-ccda-5bce-917c-2eebd553f22f
    version:      0

In [18]:
# Electron temperature
loaded_datasets["ayc"]["te"]

<xarray.DataArray 'te' (time: 73, radial_index: 131)> Size: 38kB
dask.array<open_dataset-te, shape=(73, 131), dtype=float32, chunksize=(73, 131), chunktype=numpy.ndarray>
Coordinates:
  * radial_index  (radial_index) float32 524B 0.0 1.0 2.0 ... 128.0 129.0 130.0
  * time          (time) float32 292B 0.004166 0.008332 0.0125 ... 0.3 0.3042
Attributes: (12/18)
    description:  Electron Temperature
    dims:         ['time', 'radial_index']
    file_name:    None
    format:       None
    label:        electron temperature
    mds_name:     \TOP.ANALYSED.AYC:TE
    ...           ...
    source:       ayc
    time_index:   0
    uda_name:     AYC_TE
    units:        eV
    uuid:         54a16683-8517-5f0e-a242-f36d4fbfb9ba
    version:      0

In [ ]:
# internal inductance $l_i$


In [ ]:
# radiated power (bolometer)

## Data in level 2

Download using the following command:
```bash
./s5cmd --no-sign-request --endpoint-url https://s3.echo.stfc.ac.uk cp "s3://mast/level2/shots/30420.zarr/*" ./30420.zarr/level2/


In [21]:
! ls "/home/hell/Desktop/UKAEA/Projects/1. 2024/1. Dec/1. Disruptions/bulk download/30420_level2.zarr"

control  equilibrium  mirnov	  soft_x_rays  thomson_scattering
dalpha	 magnetics    mirnov_fft  summary


In [22]:
! tree "/home/hell/Desktop/UKAEA/Projects/1. 2024/1. Dec/1. Disruptions/bulk download/30420_level2.zarr" -L 2

/home/hell/Desktop/UKAEA/Projects/1. 2024/1. Dec/1. Disruptions/bulk download/30420_level2.zarr
├── control
│   ├── gas
│   ├── gas_channel
│   ├── ref_plasma_current
│   ├── ref_plasma_density
│   ├── time
│   └── vertical_control
├── dalpha
│   ├── dalpha_mid_plane_center
│   ├── dalpha_mid_plane_wide
│   ├── dalpha_tangential
│   └── time
├── equilibrium
│   ├── beta_normal
│   ├── beta_pol
│   ├── beta_tor
│   ├── bphi_rmag
│   ├── bvac_rmag
│   ├── da_rating
│   ├── elongation
│   ├── elongation_axis
│   ├── ip_rating
│   ├── lcfs_r
│   ├── lcfs_z
│   ├── li
│   ├── magnetic_axis_r
│   ├── magnetic_axis_z
│   ├── magnetic_flux
│   ├── major_radius
│   ├── minor_radius
│   ├── n_boundary_coords
│   ├── n_x_points
│   ├── plasma_current
│   ├── profile_r
│   ├── q
│   ├── q100
│   ├── q90
│   ├── q95
│   ├── q_axis
│   ├── rpsi100_in
│   ├── rpsi100_out
│   ├── rpsi90_in
│   ├── rpsi90_out
│   ├── rpsi95_in
│   ├── rpsi95_out
│   ├── time
│   ├── triangularity_lower
│   ├── triangul

## Exploring level 2 data

In [32]:
# Set up the S3 storage options
shot_id = 30420
endpoint = "https://s3.echo.stfc.ac.uk"
url = f's3://mast/level2/shots/{shot_id}.zarr'
fs = fsspec.filesystem(
  **dict(
    protocol='simplecache',
    target_protocol="s3",
    target_options=dict(anon=True, endpoint_url=endpoint)
  )
)

filename = "s3://mast/level2/shots/"+ str(shot_number) + ".zarr"
store = zarr.storage.FSStore(fs=fs, url=url)

In [35]:

datasets = {
    "control": "control",
    "summary": "summary",
    "magnetics": "magnetics",
    "dalpha": "dalpha",
    "soft_x_rays": "soft_x_rays",
    "thomson_scattering": "thomson_scattering",

}

loaded_datasets_level2 = {}

for name, group in datasets.items():
    try:
        loaded_datasets_level2[name] = xr.open_zarr(store, group)
        print(f"Successfully loaded: {name}")
    except Exception as e:
        print(f"Error loading '{name}': {e}")

Successfully loaded: control
Successfully loaded: summary
Successfully loaded: magnetics
Successfully loaded: dalpha
Successfully loaded: soft_x_rays
Successfully loaded: thomson_scattering


In [43]:
loaded_datasets_level2[datasets["control"]]


<xarray.Dataset> Size: 336kB
Dimensions:             (gas_channel: 20, time: 1743)
Coordinates:
  * gas_channel         (gas_channel) <U16 1kB '/xdc/gas/f/bc11' ... '/xdc/ga...
  * time                (time) float64 14kB -0.0568 -0.05655 ... 0.3784 0.3787
Data variables:
    gas                 (gas_channel, time) float64 279kB dask.array<chunksize=(10, 1743), meta=np.ndarray>
    ref_plasma_current  (time) float64 14kB dask.array<chunksize=(1743,), meta=np.ndarray>
    ref_plasma_density  (time) float64 14kB dask.array<chunksize=(1743,), meta=np.ndarray>
    vertical_control    (time) float64 14kB dask.array<chunksize=(1743,), meta=np.ndarray>

In [44]:
loaded_datasets_level2[datasets["summary"]]                                                             

<xarray.Dataset> Size: 125kB
Dimensions:                      (time: 1743)
Coordinates:
  * time                         (time) float64 14kB -0.0568 -0.05655 ... 0.3787
Data variables:
    gas_inboard_total            (time) float64 14kB dask.array<chunksize=(1743,), meta=np.ndarray>
    gas_outboard_total           (time) float64 14kB dask.array<chunksize=(1743,), meta=np.ndarray>
    gas_pressure                 (time) float64 14kB dask.array<chunksize=(1743,), meta=np.ndarray>
    integrated_electron_density  (time) float64 14kB dask.array<chunksize=(1743,), meta=np.ndarray>
    line_averaged_density        (time) float64 14kB dask.array<chunksize=(1743,), meta=np.ndarray>
    neutron_flux                 (time) float64 14kB dask.array<chunksize=(1743,), meta=np.ndarray>
    plasma_current               (time) float64 14kB dask.array<chunksize=(1743,), meta=np.ndarray>
    sol_current                  (time) float64 14kB dask.array<chunksize=(1743,), meta=np.ndarray>

In [45]:
loaded_datasets_level2[datasets["magnetics"]]

<xarray.Dataset> Size: 476kB
Dimensions:                 (center_column_channel: 5, time: 1743,
                             coil_currents_channel: 6,
                             coil_voltages_channel: 4, flux_loops_channel: 6,
                             outer_discrete_channel: 8, saddle_coils_channel: 4)
Coordinates:
  * center_column_channel   (center_column_channel) <U11 220B 'xma/ccbv/04' ....
  * coil_currents_channel   (coil_currents_channel) <U20 480B 'AMC_P3L COIL C...
  * coil_voltages_channel   (coil_voltages_channel) <U12 192B '/xdc/pf/f/p1' ...
  * flux_loops_channel      (flux_loops_channel) <U12 288B 'AMB_FL/CC03' ... ...
  * outer_discrete_channel  (outer_discrete_channel) <U10 320B 'xma/obr/03' ....
  * saddle_coils_channel    (saddle_coils_channel) <U15 240B 'xmb/sad/out/m01...
  * time                    (time) float64 14kB -0.0568 -0.05655 ... 0.3787
Data variables:
    center_column           (center_column_channel, time) float64 70kB dask.array<chunksize=(5, 1743), meta=np.ndarray>
    coil_currents           (coil_currents_channel, time) float64 84kB dask.array<chunksize=(6, 1743), meta=np.ndarray>
    coil_voltages           (coil_voltages_channel, time) float64 56kB dask.array<chunksize=(4, 1743), meta=np.ndarray>
    flux_loops              (flux_loops_channel, time) float64 84kB dask.array<chunksize=(6, 1743), meta=np.ndarray>
    outer_discrete          (outer_discrete_channel, time) float64 112kB dask.array<chunksize=(8, 1743), meta=np.ndarray>
    saddle_coils            (saddle_coils_channel, time) float64 56kB dask.array<chunksize=(4, 1743), meta=np.ndarray>

In [46]:
loaded_datasets_level2[datasets["dalpha"]]

<xarray.Dataset> Size: 697kB
Dimensions:                  (time: 21780)
Coordinates:
  * time                     (time) float64 174kB -0.0568 -0.05678 ... 0.3788
Data variables:
    dalpha_mid_plane_center  (time) float64 174kB dask.array<chunksize=(21780,), meta=np.ndarray>
    dalpha_mid_plane_wide    (time) float64 174kB dask.array<chunksize=(21780,), meta=np.ndarray>
    dalpha_tangential        (time) float64 174kB dask.array<chunksize=(21780,), meta=np.ndarray>

In [47]:
loaded_datasets_level2[datasets["soft_x_rays"]]

<xarray.Dataset> Size: 96MB
Dimensions:         (hcam_l_channel: 18, time: 217800, hcam_u_channel: 18,
                     tcam_channel: 18)
Coordinates:
  * hcam_l_channel  (hcam_l_channel) <U14 1kB '/xsx/HCAM/L/1' ... '/xsx/HCAM/...
  * hcam_u_channel  (hcam_u_channel) <U14 1kB '/xsx/HCAM/U/1' ... '/xsx/HCAM/...
  * tcam_channel    (tcam_channel) <U12 864B '/xsx/TCAM/1' ... '/xsx/TCAM/9'
  * time            (time) float64 2MB -0.0568 -0.0568 -0.0568 ... 0.3788 0.3788
Data variables:
    hcam_l          (hcam_l_channel, time) float64 31MB dask.array<chunksize=(3, 27225), meta=np.ndarray>
    hcam_u          (hcam_u_channel, time) float64 31MB dask.array<chunksize=(3, 27225), meta=np.ndarray>
    tcam            (tcam_channel, time) float64 31MB dask.array<chunksize=(3, 27225), meta=np.ndarray>

In [48]:
loaded_datasets_level2[datasets["thomson_scattering"]]

<xarray.Dataset> Size: 257kB
Dimensions:       (major_radius: 120, time: 88)
Coordinates:
  * major_radius  (major_radius) float64 960B 0.3 0.31 0.32 ... 1.47 1.48 1.49
  * time          (time) float64 704B -0.0568 -0.0518 -0.0468 ... 0.3732 0.3782
Data variables:
    ne            (time, major_radius) float64 84kB dask.array<chunksize=(88, 120), meta=np.ndarray>
    ne_core       (time) float64 704B dask.array<chunksize=(88,), meta=np.ndarray>
    pe            (time, major_radius) float64 84kB dask.array<chunksize=(88, 120), meta=np.ndarray>
    te            (time, major_radius) float64 84kB dask.array<chunksize=(88, 120), meta=np.ndarray>
    te_core       (time) float64 704B dask.array<chunksize=(88,), meta=np.ndarray>